In [36]:
from plot import *
from maths import rmse


In [26]:
def _dft3_at_time(data: np.ndarray, quantile: float = 0.75):
    zf = np.fft.ifftshift(data)
    zf = np.fft.fftn(zf)
    zf = np.fft.fftshift(zf)
    amplitudes = np.abs(zf)

    zf_real = (zf.real / 512).astype("float16")
    zf_imag = (zf.imag / 512).astype("float16")

    cutoff_amp = np.quantile(amplitudes, quantile)
    transform = {(i, j, k): (zf_real[i, j, k], zf_imag[i, j, k]) for k in range(576) for j in range(361) for i in range(72)
                 if amplitudes[i, j, k] > cutoff_amp}

    return transform


In [33]:
data = load_variable_at_time("MERRA2_100.tavg3_3d_asm_Nv.19800101.nc4", "U",
                             time=0)
transform = _dft3_at_time(data, quantile=0.99)
print(len(transform))

149714


In [34]:
print(f"Size: {len(transform) * 7 / (1024 ** 2)} mB")

Size: 0.9994487762451172 mB


In [35]:
prediction = np.zeros((72, 361, 576), dtype="complex64")

for (i, j, k), val in transform.items():
    prediction[i, j, k] = val

prediction = np.fft.ifftshift(prediction)
prediction = np.fft.ifftn(prediction)
prediction = np.fft.fftshift(prediction)
prediction = prediction.real

rmse(data, prediction)


0.8210562778271889